<a href="https://colab.research.google.com/github/wenqiglantz/llmops/blob/main/FineTuning_Dataset_DataBricks_Mistral7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare dataset from databricks/databricks-dolly-15k for fine-tuning Mistral 7B

The notebook loads the Hugging Face dataset `databricks/databricks-dolly-15k`. It shuffles it and slices it to take 1000 records, transforms them into instruction dataset to fine-tune Mistral 7B.

In [ ]:
!pip install -q datasets
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: wri

In [ ]:
from datasets import load_dataset
import re, json

# Load the dataset
dataset = load_dataset('databricks/databricks-dolly-15k')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))
print(len(dataset))

# Define a function to transform the data
def transform_conversation(example):

    # Directly access the data from the example
    instruction = example.get('instruction', '')
    response = example.get('response', '')

    # Convert the input data to the desired format
    formatted_output = f"<s>[INST] {instruction} [/INST] {response}</s>\n"

    return {'text': ''.join(formatted_output)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)


1000


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
!pip install -q huggingface_hub
from huggingface_hub import create_repo , HfApi
from google.colab import userdata

username = "wenqiglantz"
dataset_name = "databricks-dolly-1k"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("HF_TOKEN"))

# Create empty dataset repo
api.create_repo(
    repo_id = f"{username}/{dataset_name}",
    repo_type="dataset",
    exist_ok=True,
)

RepoUrl('https://huggingface.co/datasets/wenqiglantz/databricks-dolly-1k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='wenqiglantz/databricks-dolly-1k')

In [ ]:
transformed_dataset.push_to_hub(f"{username}/{dataset_name}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/wenqiglantz/databricks-dolly-1k/commit/e37cf3e3db9ced17bf287ffdfdffade56dbf9543', commit_message='Upload dataset', commit_description='', oid='e37cf3e3db9ced17bf287ffdfdffade56dbf9543', pr_url=None, pr_revision=None, pr_num=None)